In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil

from rope.base.oi.soa import analyze_module
from scipy.stats import linregress
import libraries as lib

In [ ]:
directory = "../../../OneDrive - HvA/Logfiles/H2A/2022/20220628-20220701_race_nogaro/0023 - 20220701T141014 1h8m8s 13.336km Nogaro/"
file = "1.csv"

In [ ]:
data_file = lib.DataInladen(f'{directory}{file}')

In [ ]:
data_file

In [ ]:
# Filtering the data so it's only the laps we want to analyze
# First filter so that the data without GPS data is removed
data_file = data_file[data_file[xas] != 0]
data_file = data_file[data_file[yas] != 0]

# Now filter out the beginning and end of the data, by removing the first and last 10% of the data
from math import ceil
data_file
data_file = data_file.iloc[ceil(len(data_file)*0.215):ceil(len(data_file)*0.267)]

# Display the filtered data
data_file

In [ ]:
'Plotting the race'
plt.figure(figsize=(6,6))
plt.scatter(data_file['Latitude'], data_file['Longitude'], c=data_file['Snelheid over de grond in km/h'], cmap='viridis')
plt.title('GPS data vs speed')
plt.xlabel('Latitude')
plt.ylabel('Longitude')

# Adding a colorbar with a label for the speed in km/h
cbar = plt.colorbar()
cbar.set_label('Speed (km/h)')
plt.grid()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract data for plotting
latitudes = data_file['Latitude'].values
longitudes = data_file['Longitude'].values
speeds = data_file['Snelheid over de grond in km/h'].values

# Plotting the race with a 3D scatter plot
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with height representing speed
scatter = ax.scatter(latitudes, longitudes, speeds, c=speeds, cmap='viridis', s=10)

# Adding labels and title
ax.set_title('GPS data vs speed')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.set_zlabel('Speed (km/h)')

# Adding a colorbar with a label for the speed in km/h
cbar = plt.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('Speed (km/h)')

plt.show()